## ツールやライブラリ（LangChain）との統合を用いた計装

必要なライブラリをダウンロードします。

In [ ]:
%pip install -r ../requirements.txt

実行に必要な環境変数を読み込みます。

In [2]:
import os, warnings, time

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://localhost:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

Langfuseが提供するLangChainのCallback実装を初期化します。

In [3]:
from langfuse.callback import CallbackHandler

# 環境変数（LANGFUSE_SECRET_KEY, LANGFUSE_PUBLIC_KEY, LANGFUSE_HOST）を定義でも良い
callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint
)

簡易的なRAGアプリケーションを構築します。詳細は、[chapter03-03-simple-llm-app-with-rag.ipynb](./chapter03-03-simple-llm-app-with-rag.ipynb)を参照ください。

In [4]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

xql_doc = WebBaseLoader("https://techbookfest.org/product/vwEgK9fAmzRphNukv4E83P?productVariantID=b6iAh0AVyEs4hCUczPiy").load()

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.embeddings import OpenAIEmbeddings
    openai_api_key = os.getenv("OPENAI_API_KEY")
    embeddings = OpenAIEmbeddings(api_key=openai_api_key)
else:
    from langchain_cohere.embeddings import CohereEmbeddings
    cohere_api_key = os.getenv("COHERE_API_KEY")
    embeddings = CohereEmbeddings(
        cohere_api_key=cohere_api_key,
        model="embed-multilingual-v3.0"
    )

xql_db = FAISS.from_documents(
    documents=xql_doc,
    embedding=embeddings
)
xql_retriever = xql_db.as_retriever()

prompt_template = PromptTemplate.from_template(template="""
あなたは有能なアシスタントです。
以下のコンテキストに基づいて質問に対する回答を作成してください。
分からない場合は分からないと回答してください。

## コンテキスト

{context}

## 質問

{question}
""")

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    chat = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    chat = ChatCohere(cohere_api_key=cohere_api_key, model=model)

chain = (
    {"question": RunnablePassthrough(), "context": xql_retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)
# RAGを行うChainを定義
response = chain.invoke(
    input="xQL完全ガイドってなに？",
    # LangChainのCallback実装をChain実行時のConfigに渡す
    config={"callbacks": [callback_handler]}
)

print(response)

USER_AGENT environment variable not set, consider setting it to identify your requests.


xQL完全ガイドは、Prometheus、Grafana Loki、Grafana Tempo などのモニタリングツールで使用されるクエリ言語、PromQL、LogQL、TraceQL について解説したガイドブックです。これらのツールで計測したテレメトリデータから必要な情報を引き出すためのクエリの書き方を学ぶことができます。ガイドブックには、各クエリ言語の解説だけでなく、サンプルアプリや Docker 環境も用意されており、実際に手を動かしながら学ぶことができます。
